In [8]:
!pip install xlsxwriter pandas yfinance ipywidgets

import pandas as pd
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display
from xlsxwriter import Workbook

In [9]:
def save_output(data: any, description: str, save_path: str = None) -> None:
  if save_path:
    if isinstance(data, pd.DataFrame):
      data.to_csv(save_path, index=False)
    else:
      with open(save_path,"w") as f:
        f.write(str(data))
    print(f"{description} saved to {save_path}")
  else:
    print(f"{description}:")
    print(data)

def init_ticker(func):
  def wrapper(self, symbol: str, *args, **kwargs):
    ticker = yf.Ticker(symbol)
    return func(self, ticker, *args, **kwargs)
  return wrapper

class YFinanceUtils:

  @init_ticker
  def get_stock_data(self, ticker, start_date:str, end_date: str) -> pd.DataFrame:
    return ticker.history(start=start_date, end=end_date)

  @init_ticker
  def get_stock_dividends(self,ticker) -> pd.DataFrame:
    return ticker.dividends

  @init_ticker
  def get_income_stmt(self, ticker) -> pd.DataFrame:
    income_stmt = ticker.financials.T
    income_stmt.index.name = "Date"
    return income_stmt.reset_index()

  @init_ticker
  def get_balance_sheet(self, ticker) -> pd.DataFrame:
    balance_sheet = ticker.balance_sheet.T
    balance_sheet.index.name = "Date"
    return balance_sheet.reset_index()

  @init_ticker
  def get_cash_flow(self, ticker) -> pd.DataFrame:
    cash_flow = ticker.cashflow.T
    cash_flow.index.name = "Date"
    return cash_flow.reset_index()

  @init_ticker
  def get_analyst_recommendations(self, ticker) -> pd.DataFrame:
    return ticker.recommendations

def save_to_excel(data_sections:dict, file_name: str) -> None:
  with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
    for title, data in data_sections.items():
      if isinstance(data, pd.DataFrame):
        if title =="Stock Data":
          data = data.reset_index()
          data.rename(columns={"index": "Date"}, inplace=True)

          if pd.api.types.is_datetime64_any_dtype(data["Date"]):
            data["Date"] = data["Date"].dt.tz_localize(None)

        elif title in ["Income Statement", "Balance Sheet", "Cash Flow"]:
          data = data.set_index("Date").T.reset_index()
          data.rename(columns={"index": "Metric"}, inplace=True)

        data.to_excel(writer, sheet_name=title, index=False)

        worksheet = writer.sheets[title]
        for col_num, _ in enumerate(data.columns):
          worksheet.set_column(col_num, col_num, 20)

  print(f"Excel file saved to {file_name}")

def generate_report(symbol:str, start_date: str, end_date: str):
  utils = YFinanceUtils()
  print("Fetching Data...")
  stock_data = utils.get_stock_data(symbol, start_date, end_date)
  dividends = utils.get_stock_dividends(symbol)
  income_stmt = utils.get_income_stmt(symbol)
  balance_sheet = utils.get_balance_sheet(symbol)
  cash_flow = utils.get_cash_flow(symbol)
  analyst_recommendations = utils.get_analyst_recommendations(symbol)

  excel_data = {
      "Stock Data": stock_data,
      "Dividends": dividends,
      "Income Statement": income_stmt,
      "Balance Sheet": balance_sheet,
      "Cash Flow": cash_flow,
      "Analyst Recommendations": analyst_recommendations,
  }

  file_name = f"{symbol}_Stock_Report.xlsx"
  save_to_excel(excel_data, file_name)

def on_submit(_):
  symbol = ticker_input.value
  start_date = start_date_input.value
  end_date = end_date_input.value

  if not symbol or not start_date or not end_date:
    print("Please fill in all fields.")
    return

  generate_report(symbol, start_date, end_date)

ticker_input = widgets.Text(description="Ticker:", placeholder="e.g., AAPL")
start_date_input = widgets.Text(description="Start Date:", placeholder="YYYY-MM-DD")
end_date_input = widgets.Text(description="End Date:", placeholder="YYYY-MM-DD")
submit_button = widgets.Button(description="Generate Report")

submit_button.on_click(on_submit)

display(widgets.VBox([ticker_input, start_date_input, end_date_input, submit_button]))


Fetching Data...
Excel file saved to AAPL_Stock_Report.xlsx
